In [ ]:
import torch
import torchvision

#Criando um transformador de imgs em tensores.
#torch só lida com tensores(forma númerica de representar informações)
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

#baixar o dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

#divisão, serem processados juntos
#testar 32, 64, 128 e 256
batch_size = 64

#dividir o dataset
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

100%|██████████| 170498071/170498071 [00:04<00:00, 39402402.36it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
print(type(trainset))

<class 'torchvision.datasets.cifar.CIFAR10'>


In [ ]:
import torch.nn as nn
import torch.optim as optim

class CIFAR10Model(nn.Module):
    def __init__(self):
        super().__init__()

        #Definicão da camada convolucional
        #entrada, saida(base 2), filtro, salto, camada de 0
        self.conv1 = nn.Conv2d(3, 32, kernel_size=(3,3), stride=1, padding=1)
        #função de ativação, o neurónio vai ser ativado?
        self.act1 = nn.ReLU()
        #stride assume valor do kernel, de tamanho 2, vai diminuir pela metade a imagem
        #self.pool2 = nn.AvgPool2d(kernel_size=(2, 2))
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2))

        self.conv2 = nn.Conv2d(32, 32, kernel_size=(3,3), stride=1, padding=1)
        self.act2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2))

        #converter em um vetor uni
        self.flat = nn.Flatten()

        self.fc3 = nn.Linear(8192, 512)
        self.act3 = nn.ReLU()
        #evitar overfitting (não generalização)
        #desativar metade dos neuronios
        self.drop3 = nn.Dropout(0.5)
        #camada de entrada, de saida
        self.fc4 = nn.Linear(512, 10)

    def forward(self, x):
        #inferencia = teste
        x = self.act1(self.conv1(x))
        x = self.act2(self.conv2(x))
        x = self.pool2(x)
        x = self.flat(x)
        x = self.act3(self.fc3(x))
        x = self.drop3(x)
        x = self.fc4(x)
        return x

model = CIFAR10Model()
loss_fn = nn.CrossEntropyLoss()
#lr maior: divergir
#colocar lr maior no começo e depois diminuir?, momentum = acumular
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9)

#hodout
n_epochs = 20
for epoch in range(n_epochs):
    print(f'epoch {epoch}')
    for inputs, labels in trainloader:
        # forward, backward, and then weight update
        y_pred = model(inputs)
        loss = loss_fn(y_pred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    acc = 0
    count = 0
    for inputs, labels in testloader:
        y_pred = model(inputs)
        acc += (torch.argmax(y_pred, 1) == labels).float().sum()
        count += len(labels)
    acc /= count
    print("Epoch %d: model accuracy %.2f%%" % (epoch, acc*100))

torch.save(model.state_dict(), "cifar10model.pth")

epoch 0
Epoch 0: model accuracy 47.71%
epoch 1
Epoch 1: model accuracy 59.16%
epoch 2
Epoch 2: model accuracy 65.38%
epoch 3
Epoch 3: model accuracy 69.14%
epoch 4
Epoch 4: model accuracy 74.06%
epoch 5
Epoch 5: model accuracy 78.67%
epoch 6
Epoch 6: model accuracy 82.72%
epoch 7
Epoch 7: model accuracy 84.04%
epoch 8
Epoch 8: model accuracy 88.37%
epoch 9
Epoch 9: model accuracy 89.36%
epoch 10
Epoch 10: model accuracy 91.33%
epoch 11
Epoch 11: model accuracy 92.73%
epoch 12
Epoch 12: model accuracy 93.39%
epoch 13
Epoch 13: model accuracy 94.60%
epoch 14
Epoch 14: model accuracy 94.71%
epoch 15
Epoch 15: model accuracy 95.61%
epoch 16
Epoch 16: model accuracy 96.00%
epoch 17
Epoch 17: model accuracy 96.46%
epoch 18
Epoch 18: model accuracy 96.11%
epoch 19
Epoch 19: model accuracy 96.52%
